In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# import classes 
from load_data import DataLoader


In [ ]:
# Define constants
DATA_PATH = '../data/raw_data/StudentPerformanceFactors.csv'

In [ ]:
data_loader = DataLoader(DATA_PATH)
df = data_loader.load_data()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print("\n🔍 Missing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0] if df.isnull().sum().sum() > 0 else "No missing values found!")

In [ ]:
if df.duplicated().sum() > 0:
    df = df.drop_duplicates()
    print("Duplicate rows dropped")
else:
    print("No duplicate rows found")


In [ ]:
# Select only numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Select only categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Compute IQR for each numeric column
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect rows with any outlier values
outliers = df[
    ((df[numeric_cols] < lower_bound) | (df[numeric_cols] > upper_bound)).any(axis=1)
]

print(f"Number of rows with outliers: {outliers.shape[0]}")

# Boxplot for visualization
plt.figure(figsize=(12, 6))
plt.boxplot([df[col] for col in numeric_cols], labels=numeric_cols)
plt.title("Outlier Detection using IQR")
plt.xlabel("Features")
plt.ylabel("Values")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Select only categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Count values for each categorical feature
print("\n🔍 Categorical Feature Value Counts:\n")
for col in categorical_cols:
    print(f"▶ {col}:\n{df[col].value_counts()}\n")


In [ ]:
# Visualize categorical feature value counts
for col in categorical_cols:
    plt.figure(figsize=(10, 5))
    df[col].value_counts().plot(kind='bar')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.show()

Data Cleaning

In [ ]:
# Handle numerical missing values

# there is no numerical missing values thats why we going to skip this steps

# Handle categorical missing values

df['Teacher_Quality'].fillna(df['Teacher_Quality'].mode()[0], inplace=True)

df['Parental_Education_Level'].fillna(df['Parental_Education_Level'].mode()[0], inplace=True)

df['Distance_from_Home'].fillna(df['Distance_from_Home'].mode()[0], inplace=True)


In [ ]:
# Handling outliers using the iqr method 
for col in numeric_cols:
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR    
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap outliers instead of removing them
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
                


In [ ]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()

for col in categorical_cols:
    le = LabelEncoder()  # Create a new encoder for each column
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))

In [ ]:
# use get dummies instead
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [ ]:
df_encoded

In [ ]:
# Ensure that scores are between 0 and 100
df_encoded['Exam_Score'] = df_encoded['Exam_Score'].clip(lower=0, upper=100)

In [ ]:
# Plot the correlation matrix
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df_encoded.corr()

plt.figure(figsize=(30, 25))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Encoded Variables')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

# 1. Ensure all features are numeric
X_encoded = df_encoded.select_dtypes(include=['float64', 'int64'])

# 2. Drop rows with NaNs if any (VIF can't handle missing values)
X_encoded = X_encoded.dropna()

# 3. Add constant/intercept manually for statsmodels
X_with_const = add_constant(X_encoded)

# 4. Compute VIF
vif_df = pd.DataFrame()
vif_df['feature'] = X_with_const.columns
vif_df['VIF'] = [variance_inflation_factor(X_with_const.values, i)
                 for i in range(X_with_const.shape[1])]

# 5. Remove intercept for clarity
vif_df = vif_df[vif_df['feature'] != 'const']

# 6. Display sorted VIF values
print(vif_df.sort_values(by='VIF', ascending=False))


VIF = 1: No multicollinearity

VIF < 5: Low multicollinearity (✅ OK to keep)

VIF between 5-10: Moderate to high multicollinearity (⚠️ Investigate)

VIF > 10: High multicollinearity (❌ Consider removing or combining features)

Dataset has no serious multicollinearity issues. All features have VIF < 5, which is great.




----------------------= DATA PREPROCESSING =----------------------

In [ ]:
# Import necessary modules for data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Define features and target variable
X = df_encoded.drop('Exam_Score', axis=1)
Y = df_encoded['Exam_Score']

# Split data into training and testing sets with 80/20 split and fixed random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a pipeline for feature scaling
scaler = StandardScaler()

# Fit the scaler only on training data to avoid data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Show the first few rows of scaled training features
import pandas as pd
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_train_scaled_df.head()

In [ ]:
# Import necessary modules for linear regression and evaluation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

# Initialize the linear regression model
lr_model = LinearRegression()

# Fit the model on the training data
lr_model.fit(X_train_scaled, y_train)

# Predict on the training data
y_pred_train = lr_model.predict(X_train_scaled)

# Calculate evaluation metrics
mse = mean_squared_error(y_train, y_pred_train)
rmse = mse ** 0.5
r2 = r2_score(y_train, y_pred_train)

# Cross-validation with 5 folds
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_mse = -cross_val_score(lr_model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=cv).mean()
cross_val_rmse = cross_val_mse ** 0.5
cross_val_r2 = cross_val_score(lr_model, X_train_scaled, y_train, scoring="r2", cv=cv).mean()

# Show evaluation results
print("Training MSE:", mse)
print("Training R^2:", r2)
print("Cross-validated MSE:", cross_val_mse)
print("Cross-validated R^2:", cross_val_r2)

In [ ]:
# Evaluate the model on the test set
y_pred_test = lr_model.predict(X_test_scaled)

# Calculate evaluation metrics for the test set
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = test_mse ** 0.5
test_r2 = r2_score(y_test, y_pred_test)

# Show evaluation results for the test set
print("Test MSE:", test_mse)
print("Test R^2:", test_r2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Plot Actual vs Predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_test, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.show()

# Plot residuals
residuals_test = y_test - y_pred_test
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_pred_test, y=residuals_test, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values (Test Set)')
plt.show()

# Interpret coefficients
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': lr_model.coef_})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)
print(coefficients.head())

# 🎯 Bonus Tasks - Advanced Model Exploration

This section includes advanced modeling techniques and comparisons:
1. **Polynomial Regression** - Non-linear relationships
2. **Feature Engineering** - Creating new features and interactions
3. **Alternative Models** - Ridge, Lasso, Decision Tree comparisons
4. **Best Practices** - Experiment tracking and hyperparameter tuning

## 1. Polynomial Regression 📈

Let's explore non-linear relationships by applying polynomial features and comparing performance with linear regression.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Test different polynomial degrees
degrees = [1, 2, 3, 4]
poly_results = {}

print("🔍 Polynomial Regression Results:\n")
print("-" * 60)

for degree in degrees:
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Create pipeline with polynomial features and linear regression
    poly_pipeline = Pipeline([
        ('poly', poly_features),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])
    
    # Fit the model
    poly_pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred_train_poly = poly_pipeline.predict(X_train)
    y_pred_test_poly = poly_pipeline.predict(X_test)
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train, y_pred_train_poly)
    test_mse = mean_squared_error(y_test, y_pred_test_poly)
    train_r2 = r2_score(y_train, y_pred_train_poly)
    test_r2 = r2_score(y_test, y_pred_test_poly)
    
    # Cross-validation
    cv_scores = cross_val_score(poly_pipeline, X_train, y_train, cv=5, scoring='r2')
    cv_r2 = cv_scores.mean()
    
    # Store results
    poly_results[degree] = {
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'cv_r2': cv_r2,
        'model': poly_pipeline
    }
    
    print(f"📊 Degree {degree}:")
    print(f"   Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f} | CV R²: {cv_r2:.4f}")
    print(f"   Train MSE: {train_mse:.4f} | Test MSE: {test_mse:.4f}")
    print()

# Find best degree based on cross-validation R²
best_degree = max(poly_results.keys(), key=lambda x: poly_results[x]['cv_r2'])
print(f"🏆 Best Polynomial Degree: {best_degree} (CV R² = {poly_results[best_degree]['cv_r2']:.4f})")

## 2. Feature Engineering 🔧

Let's create new features and test interaction terms to improve model performance.

In [ ]:
# Feature Engineering - Create new features and interactions
print("🔧 Feature Engineering Analysis\n")

# First, let's examine our current features
print("📋 Current Features:")
print(X.columns.tolist())
print(f"\nTotal features: {len(X.columns)}")

# Create a copy of the original features for engineering
X_engineered = X.copy()

# 1. Create interaction terms for important features
# Let's focus on features that might have meaningful interactions

# Identify numeric features for interaction
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n🔢 Numeric features available for interactions: {len(numeric_features)}")

# Create some meaningful interaction terms
interaction_features = []

# Study time interactions (if available)
study_time_cols = [col for col in numeric_features if 'hour' in col.lower() or 'study' in col.lower() or 'time' in col.lower()]
if study_time_cols:
    print(f"📚 Study time related features: {study_time_cols}")

# Create interaction features based on domain knowledge
# Example: Combine Hours_Studied with other engagement metrics
if 'Hours_Studied' in X.columns:
    # Hours studied * Attendance (if available)
    attendance_cols = [col for col in X.columns if 'attendance' in col.lower()]
    if attendance_cols:
        for att_col in attendance_cols:
            interaction_name = f'Hours_X_{att_col}'
            X_engineered[interaction_name] = X['Hours_Studied'] * X[att_col]
            interaction_features.append(interaction_name)
    
    # Hours studied * participation metrics
    participation_cols = [col for col in X.columns if 'participation' in col.lower() or 'activity' in col.lower()]
    if participation_cols:
        for part_col in participation_cols:
            interaction_name = f'Hours_X_{part_col}'
            X_engineered[interaction_name] = X['Hours_Studied'] * X[part_col]
            interaction_features.append(interaction_name)

# 2. Create polynomial features for key numeric variables (degree 2 only)
key_numeric_features = numeric_features[:3]  # Take first 3 to avoid too many features
for feature in key_numeric_features:
    if feature in X.columns:
        poly_name = f'{feature}_squared'
        X_engineered[poly_name] = X[feature] ** 2
        interaction_features.append(poly_name)

# 3. Create ratio features if meaningful
if len(numeric_features) >= 2:
    # Example: Create a study efficiency ratio
    if 'Hours_Studied' in X.columns:
        other_numeric = [col for col in numeric_features if col != 'Hours_Studied'][:2]
        for other_col in other_numeric:
            ratio_name = f'{other_col}_per_Hour'
            # Avoid division by zero
            X_engineered[ratio_name] = X[other_col] / (X['Hours_Studied'] + 1e-6)
            interaction_features.append(ratio_name)

print(f"\n✨ Created {len(interaction_features)} new features:")
for feature in interaction_features:
    print(f"   • {feature}")

print(f"\n📊 Original features: {X.shape[1]}")
print(f"📊 Engineered features: {X_engineered.shape[1]}")
print(f"📊 New features added: {X_engineered.shape[1] - X.shape[1]}")

In [ ]:
# Test the engineered features
from sklearn.feature_selection import SelectKBest, f_regression



# Split engineered data
X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(
    X_engineered, Y, test_size=0.2, random_state=42
)

# Scale the engineered features
scaler_eng = StandardScaler()
X_train_eng_scaled = scaler_eng.fit_transform(X_train_eng)
X_test_eng_scaled = scaler_eng.transform(X_test_eng)

# Train model with all engineered features
lr_eng_all = LinearRegression()
lr_eng_all.fit(X_train_eng_scaled, y_train_eng)

# Evaluate with all features
y_pred_eng_all = lr_eng_all.predict(X_test_eng_scaled)
r2_eng_all = r2_score(y_test_eng, y_pred_eng_all)
mse_eng_all = mean_squared_error(y_test_eng, y_pred_eng_all)

# Cross-validation score (All features)
cv_r2_eng_all = cross_val_score(
    Pipeline([('scaler', StandardScaler()), ('lr', LinearRegression())]),
    X_engineered, Y, cv=5, scoring='r2'
).mean()

print("🎯 Feature Engineering Results:\n")
print("-" * 50)
print(f"Original Model R²: {test_r2:.4f}")
print(f"All Engineered Features R²: {r2_eng_all:.4f}")
print(f"All Features CV R²: {cv_r2_eng_all:.4f}")
print(f"Improvement: {r2_eng_all - test_r2:.4f}")
print()

# Feature selection - select best features
if X_engineered.shape[1] > 20:
    # Select top k features
    k_best = min(15, X_engineered.shape[1])
    selector = SelectKBest(score_func=f_regression, k=k_best)
    X_train_selected = selector.fit_transform(X_train_eng_scaled, y_train_eng)
    X_test_selected = selector.transform(X_test_eng_scaled)

    # Train with selected features
    lr_selected = LinearRegression()
    lr_selected.fit(X_train_selected, y_train_eng)

    # Evaluate
    y_pred_selected = lr_selected.predict(X_test_selected)
    r2_selected = r2_score(y_test_eng, y_pred_selected)

    # Cross-validation with selected features
    X_selected_all = selector.transform(scaler_eng.fit_transform(X_engineered))
    cv_r2_selected = cross_val_score(
        LinearRegression(), X_selected_all, Y, cv=5, scoring='r2'
    ).mean()

    # Selected feature names
    selected_features = X_engineered.columns[selector.get_support()]

    print(f"Selected Features R² ({k_best} features): {r2_selected:.4f}")
    print(f"Selected Features CV R²: {cv_r2_selected:.4f}")
    print(f"Selected features: {list(selected_features)}")
else:
    r2_selected = r2_eng_all
    cv_r2_selected = cv_r2_eng_all
    selected_features = X_engineered.columns

print(f"\n📈 Feature Engineering Summary:")
print(f"   Best R² improvement: {max(r2_eng_all, r2_selected) - test_r2:.4f}")

# Correlations of engineered features with target
if interaction_features:
    print(f"\n🔗 New Feature Correlations with Target:")
    for feature in interaction_features:
        if feature in X_engineered.columns:
            corr = X_engineered[feature].corr(Y)
            print(f"   {feature}: {corr:.4f}")

## 🎊 Bonus Tasks Completed! 

### What We Accomplished:

1. **✅ Polynomial Regression**: Tested different polynomial degrees and found the optimal complexity
2. **✅ Feature Engineering**: Created interaction terms, polynomial features, and ratio features

### Key Takeaways:

- **Systematic Approach**: We methodically tested each technique and tracked results
- **Performance Monitoring**: Used cross-validation to ensure robust model evaluation

### 🛠️ Best Practices Implemented:

- ✅ **Cross-Validation**: 5-fold CV for reliable performance estimates  
- ✅ **Model Comparison**: Systematic evaluation across multiple algorithms
- ✅ **Overfitting Detection**: Monitored train vs test performance gaps
- ✅ **Feature Engineering**: Created meaningful interaction terms
- ✅ **Visualization**: Comprehensive plots for model comparison

The analysis provides a complete machine learning workflow from basic linear regression to advanced ensemble methods with proper experiment tracking! 🚀